# Scaling Multi-Objective Bayesian Optimisation using RayTune

This notebook will explore various Multi-Objective Bayesian Optimisation algorithms and evaluate their distributed schedulers for fast, efficient scaling. The task at hand is Neural Architecture Search (NAS), and the following auxiliary frameworks will be used:

1. **Ax**: Abstraction on top of the popular BoTorch library
2. **Optuna**: Uses tree-structured Parzen Estimator (TPE) for Bayesian optimisation

**RayTune** is the scalable optimisation framework that acts as a distributed backbone for these auxiliary algorithms, and helps scale them in distributed environments. The notebook compares their performance on NAS tasks of varying performances, and in particular, probes their scheduling algorithms that control execution and search speed.

In [1]:
import time
import ray
from ray import train, tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.search.optuna import OptunaSearch

## Neural Architecture Search (NAS)

NAS is a high-level optimisation problem that aims to find the most optimal neural network architecture for specific tasks. Naturally, there are several considerations when designing a neural network, such as the tradeoff between model size and performance and the computational complexity of layers. A large search space requires *multi-objective* optimisation, with Bayesian methods being particularly useful because it is infeasible to train and evaluate every possible architecture. 

### With Ax

In [2]:
from dataclasses import dataclass
import tempfile
from filelock import FileLock
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, random_split
from ray.train.lightning import (
    RayDDPStrategy,
    RayLightningEnvironment,
    RayTrainReportCallback,
    prepare_trainer,
)
from torchmetrics import Accuracy


@dataclass
class ModelConfig:
    learning_rate: float
    batch_size: int
    layer_1_size: int
    layer_2_size: int
    layer_3_size: int
    dropout: float


class MNISTClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.accuracy = Accuracy(task="multiclass", num_classes=10, top_k=1)

        # model parameters
        self.layer1 = nn.Linear(28 * 28, config.layer_1_size)
        self.layer2 = nn.Linear(config.layer_1_size, config.layer_2_size)
        self.layer3 = nn.Linear(config.layer_2_size, config.layer_3_size)
        self.layer4 = nn.Linear(config.layer_3_size, 10)
        self.dropout = nn.Dropout(config.dropout)

        # training parameters
        self.learning_rate = config.learning_rate

        self.eval_loss = []
        self.eval_accuracy = []

    def forward(self, x):
        batch_size, channels, width, height = x.size()

        x = x.view(batch_size, -1)

        x = F.relu(self.layer1(x))
        x = self.dropout(x)

        x = F.relu(self.layer2(x))
        x = self.dropout(x)

        x = F.relu(self.layer3(x))
        x = self.dropout(x)

        x = self.layer4(x)
        x = torch.log_softmax(x, dim=1)
        return x

    def cross_entropy_loss(self, logits, labels):
        """
        Apply NLL loss because softmax is applied in the forward function
        """
        return F.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        accuracy = self.accuracy(logits, y)

        self.log("ptl/train_loss", loss)
        self.log("ptl/train_accuracy", accuracy)

        return loss

    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        accuracy = self.accuracy(logits, y)

        self.eval_loss.append(loss)
        self.eval_accuracy.append(accuracy)

        return {"val_loss": loss, "val_accuracy": accuracy}

    def on_validation_epoch_end(self):
        avg_loss = torch.stack(self.eval_loss).mean()
        avg_acc = torch.stack(self.eval_accuracy).mean()
        self.log("ptl/val_loss", avg_loss, sync_dist=True)
        self.log("ptl/val_accuracy", avg_acc, sync_dist=True)
        self.eval_loss.clear()
        self.eval_accuracy.clear()

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer


class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=128):
        super().__init__()
        self.data_dir = tempfile.mkdtemp()
        self.batch_size = batch_size
        self.transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )

    def setup(self, stage=None):
        with FileLock(f"{self.data_dir}.lock"):
            mnist = MNIST(
                self.data_dir, train=True, download=True, transform=self.transform
            )
            self.mnist_train, self.mnist_val = random_split(mnist, [55000, 5000])
            self.mnist_test = MNIST(
                self.data_dir, train=False, download=True, transform=self.transform
            )

    def train_dataloader(self):
        return DataLoader(self.mnist_train, 
                          self.batch_size, num_workers=4)

    def val_dataloader(self):
        return DataLoader(self.mnist_val, batch_size=self.batch_size, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.mnist_test, batch_size=self.batch_size, num_workers=4)


def train_func(config):
    print("printing config", config)
    with open("config.txt", "w") as f:
        f.write(str(config))
    data_module = MNISTDataModule(batch_size=config["batch_size"])

    # instantiate config object
    config = ModelConfig(
        learning_rate=config["learning_rate"],
        batch_size=config["batch_size"],
        layer_1_size=config["layer_1_size"],
        layer_2_size=config["layer_2_size"],
        layer_3_size=config["layer_3_size"],
        dropout=config["dropout"],
    )

    model = MNISTClassifier(config=config)

    trainer = pl.Trainer(
        devices=1,
        accelerator="cpu",
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
        enable_progress_bar=False,
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, datamodule=data_module)

In [3]:
search_space = {
    "layer_1_size": tune.choice([32, 64, 128]),
    "layer_2_size": tune.choice([64, 128, 256]),
    "layer_3_size": tune.choice([128, 256, 512]),
    "dropout": tune.uniform(0.1, 0.3),
    "batch_size": tune.choice([32, 64, 128]),
    "learning_rate": tune.loguniform(1e-4, 1e-1),
}

In [4]:
from ray.train import RunConfig, ScalingConfig, CheckpointConfig

# scaling_config = ScalingConfig(
#     num_workers=3,
#     use_gpu=True,
#     resources_per_worker={"CPU": 1, "GPU": 1}
# )

run_config = RunConfig(
    checkpoint_config=CheckpointConfig(
        num_to_keep=2,
        checkpoint_score_attribute="ptl/val_accuracy",
        checkpoint_score_order="max"
    )
)

In [5]:
from ray.train.torch import TorchTrainer

ray_trainer = TorchTrainer(
    train_func,
    # scaling_config=scaling_config,
    run_config=run_config,
)

In [6]:
from ray.tune.schedulers import ASHAScheduler

num_epochs = 5
num_samples = 100

scheduler = ASHAScheduler(
    max_t=num_epochs,
    grace_period=1,
    reduction_factor=2
)

In [7]:
from ray.tune.search.ax import AxSearch

algorithm = AxSearch()

# restrict to 4 concurrent trials
algorithm = tune.search.ConcurrencyLimiter(algorithm, max_concurrent=4)

[WARNING 12-24 17:34:19] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [8]:
tuner = tune.Tuner(
    ray_trainer,
    param_space={"train_loop_config": search_space},
    tune_config=tune.TuneConfig(
        metric="ptl/val_accuracy",
        mode="max",
        num_samples=num_samples,
        scheduler=scheduler,
    ),
)

tuner.fit()

(TorchTrainer pid=51189) Started distributed worker processes: 
(TorchTrainer pid=51189) - (node_id=7cc2334a334327f410131b140fb9ad2ae54820f537299598f92261b4, ip=127.0.0.1, pid=51246) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=51248) Setting up process group for: env:// [rank=0, world_size=1]


(RayTrainWorker pid=51247) printing config {'layer_1_size': 64, 'layer_2_size': 256, 'layer_3_size': 256, 'dropout': 0.16465667111048166, 'batch_size': 128, 'learning_rate': 0.0012649991136170205}
(RayTrainWorker pid=51247) Downloading http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz


(RayTrainWorker pid=51248) GPU available: True (mps), used: False
(RayTrainWorker pid=51248) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=51248) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=51248) /Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
(RayTrainWorker pid=51248) /Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.


(RayTrainWorker pid=51247) Failed to download (trying next):
(RayTrainWorker pid=51247) HTTP Error 403: Forbidden
(RayTrainWorker pid=51247) 
(RayTrainWorker pid=51248) 
(RayTrainWorker pid=51246) 
(RayTrainWorker pid=51249) 
(RayTrainWorker pid=51245) 
(RayTrainWorker pid=51247) Downloading https://ossci-datasets.s3.amazonaws.com/mnist/train-images-idx3-ubyte.gz to /var/folders/mf/cfzv46p15017stkscsq9jw9h0000gn/T/tmpoka2j6og/MNIST/raw/train-images-idx3-ubyte.gz


  0%|          | 0.00/9.91M [00:00<?, ?B/s]
  1%|          | 98.3k/9.91M [00:00<00:16, 599kB/s]
 27%|██▋       | 2.72M/9.91M [00:00<00:01, 5.91MB/s]
 97%|█████████▋| 9.60M/9.91M [00:01<00:00, 10.5MB/s]
100%|██████████| 9.91M/9.91M [00:01<00:00, 8.45MB/s]
 92%|█████████▏| 9.14M/9.91M [00:01<00:00, 10.8MB/s]


...

100%|██████████| 1.65M/1.65M [00:00<00:00, 3.35MB/s]


(RayTrainWorker pid=57177) Failed to download (trying next):
(RayTrainWorker pid=57177) HTTP Error 403: Forbidden
(RayTrainWorker pid=57177) 
(RayTrainWorker pid=57177) Downloading https://ossci-datasets.s3.amazonaws.com/mnist/t10k-labels-idx1-ubyte.gz to /var/folders/mf/cfzv46p15017stkscsq9jw9h0000gn/T/tmpynugre8n/MNIST/raw/t10k-labels-idx1-ubyte.gz
(RayTrainWorker pid=57177) Extracting /var/folders/mf/cfzv46p15017stkscsq9jw9h0000gn/T/tmpynugre8n/MNIST/raw/t10k-labels-idx1-ubyte.gz to /var/folders/mf/cfzv46p15017stkscsq9jw9h0000gn/T/tmpynugre8n/MNIST/raw
(RayTrainWorker pid=57177) 


100%|██████████| 4.54k/4.54k [00:00<00:00, 1.45MB/s]
(RayTrainWorker pid=56542) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/sidharrthnagappan/ray_results/TorchTrainer_2024-12-24_17-34-19/TorchTrainer_4f4b8_00034_34_batch_size=128,dropout=0.2389,layer_1_size=128,layer_2_size=64,layer_3_size=128,learning_rate=0.0025_2024-12-24_17-34-21/checkpoint_000002)
(RayTrainWorker pid=57177) 
(RayTrainWorker pid=57177)   | Name     | Type               | Params | Mode 
(RayTrainWorker pid=57177) --------------------------------------------------------
(RayTrainWorker pid=57177) 0 | accuracy | MulticlassAccuracy | 0      | train
(RayTrainWorker pid=57177) 1 | layer1   | Linear             | 50.2 K | train
(RayTrainWorker pid=57177) 2 | layer2   | Linear             | 8.3 K  | train
(RayTrainWorker pid=57177) 3 | layer3   | Linear             | 66.0 K | train
(RayTrainWorker pid=57177) 4 | layer4   | Linear             | 5.1 K  | train
(RayTrainWorker pid=57177) 5 | d

ResultGrid<[
  Result(
    metrics={'ptl/train_loss': 0.14144454896450043, 'ptl/train_accuracy': 0.9659090638160706, 'ptl/val_loss': 0.09280373156070709, 'ptl/val_accuracy': 0.970507800579071, 'epoch': 4, 'step': 2150},
    path='/Users/sidharrthnagappan/ray_results/TorchTrainer_2024-12-24_17-34-19/TorchTrainer_4f4b8_00000_0_batch_size=128,dropout=0.1647,layer_1_size=64,layer_2_size=256,layer_3_size=256,learning_rate=0.0013_2024-12-24_17-34-21',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/Users/sidharrthnagappan/ray_results/TorchTrainer_2024-12-24_17-34-19/TorchTrainer_4f4b8_00000_0_batch_size=128,dropout=0.1647,layer_1_size=64,layer_2_size=256,layer_3_size=256,learning_rate=0.0013_2024-12-24_17-34-21/checkpoint_000004)
  ),
  Result(
    metrics={'ptl/train_loss': 0.24482308328151703, 'ptl/train_accuracy': 0.9583333134651184, 'ptl/val_loss': 0.1930762678384781, 'ptl/val_accuracy': 0.9432357549667358, 'epoch': 0, 'step': 860},
    path='/Users/sidharrthnag

(RayTrainWorker pid=56664) /Users/sidharrthnagappan/.pyenv/versions/3.11.0/lib/python3.11/multiprocessing/resource_tracker.py:224: UserWarning: resource_tracker: There appear to be 55 leaked semaphore objects to clean up at shutdown
(RayTrainWorker pid=56664)   warnings.warn('resource_tracker: There appear to be %d '


In [ ]:
tuner.results_df

In [9]:
from axsearch_multiobjective import AxSearchMultiObjective

In [1]:
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from ray import tune, air
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from ray.tune.search.ax import AxSearch
from ray.train import RunConfig, ScalingConfig, CheckpointConfig

# from ray.tune.search.ax import AxSearch
from axsearch_multiobjective import AxSearchMultiObjective


def evaluate(parameter: dict, checkpoint_dir=None):
    session.report(
        {
            "a": parameter["a"],
            "b": parameter["b"],
        }
    )


ax_client = AxClient(
    verbose_logging=False,
    # enforce_sequential_optimization=False,
)
ax_client.create_experiment(
    name="test",
    parameters=[
        {
            "name": "a",
            "type": "range",
            "value_type": "float",
            "bounds": [0, 1.0],
        },
        {
            "name": "b",
            "type": "range",
            "value_type": "float",
            "bounds": [0, 1.0],
        },
    ],
    objectives={
        "a": ObjectiveProperties(minimize=True, threshold=0.5),
        "b": ObjectiveProperties(minimize=True, threshold=0.5),
    },
    overwrite_existing_experiment=True,
    is_test=False,
)

algo = AxSearchMultiObjective(ax_client=ax_client)
algo = ConcurrencyLimiter(algo, max_concurrent=1)
tuner = tune.Tuner(
    tune.with_resources(evaluate, resources={"cpu": 1}),
    tune_config=tune.TuneConfig(search_alg=algo, num_samples=60),
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=2,
            checkpoint_score_attribute="ptl/val_accuracy",
            checkpoint_score_order="max",
        )
    ),
)
tuner.fit()

observations []
observation_features []
Suggested config: {'a': 0.20519250631332397, 'b': 0.9462937712669373}


/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))


Result: {'a': 0.20519250631332397, 'b': 0.9462937712669373, 'timestamp': 1735082657, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': '3851fd8b', 'date': '2024-12-24_23-24-17', 'time_this_iter_s': 0.0001430511474609375, 'time_total_s': 0.0001430511474609375, 'pid': 89948, 'hostname': 'Sidharrths-MacBook-Pro-142.local', 'node_ip': '127.0.0.1', 'time_since_restore': 0.0001430511474609375, 'iterations_since_restore': 1, 'experiment_tag': '1_a=0.2052,b=0.9463', 'config/a': 0.20519250631332397, 'config/b': 0.9462937712669373}
Metrics to include: ['a', 'b']
Metric dict after trial: {'a': (0.20519250631332397, None), 'b': (0.9462937712669373, None)}
observations [<ax.core.observation.Observation object at 0x165f1d050>]
observation_features [ObservationFeatures(parameters={'a': 0.20519250631332397, 'b': 0.9462937712669373}, trial_index=0)]
Suggested config: {'a': 0.9658348821103573, 'b': 0.3620511395856738}


/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))


Result: {'a': 0.9658348821103573, 'b': 0.3620511395856738, 'timestamp': 1735082658, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': '9396dabf', 'date': '2024-12-24_23-24-18', 'time_this_iter_s': 0.00010800361633300781, 'time_total_s': 0.00010800361633300781, 'pid': 89950, 'hostname': 'Sidharrths-MacBook-Pro-142.local', 'node_ip': '127.0.0.1', 'time_since_restore': 0.00010800361633300781, 'iterations_since_restore': 1, 'experiment_tag': '2_a=0.9658,b=0.3621', 'config/a': 0.9658348821103573, 'config/b': 0.3620511395856738}
Metrics to include: ['a', 'b']
Metric dict after trial: {'a': (0.9658348821103573, None), 'b': (0.3620511395856738, None)}
observations [<ax.core.observation.Observation object at 0x17f31b790>, <ax.core.observation.Observation object at 0x17f34f350>]
observation_features [ObservationFeatures(parameters={'a': 0.20519250631332397, 'b': 0.9462937712669373}, trial_index=0), ObservationFeatures(parameters={'a': 0.9658348821103573, 'b': 0.3620

...

observations [<ax.core.observation.Observation object at 0x17f31a450>, <ax.core.observation.Observation object at 0x3867d06d0>, <ax.core.observation.Observation object at 0x3868706d0>, <ax.core.observation.Observation object at 0x3867ddcd0>, <ax.core.observation.Observation object at 0x386829b50>, <ax.core.observation.Observation object at 0x38687fd10>, <ax.core.observation.Observation object at 0x38688e490>]
observation_features [ObservationFeatures(parameters={'a': 0.20519250631332397, 'b': 0.9462937712669373}, trial_index=0), ObservationFeatures(parameters={'a': 0.9658348821103573, 'b': 0.3620511395856738}, trial_index=1), ObservationFeatures(parameters={'a': 0.5993889393284917, 'b': 0.6957563571631908}, trial_index=2), ObservationFeatures(parameters={'a': 0.3543400028720498, 'b': 0.11307110730558634}, trial_index=3), ObservationFeatures(parameters={'a': 0.38054878637194633, 'b': 0.5852137189358473}, trial_index=4), ObservationFeatures(parameters={'a': 0.0, 'b': 0.0}, trial_index=5)

/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


observations [<ax.core.observation.Observation object at 0x38679aa10>, <ax.core.observation.Observation object at 0x3867f5350>, <ax.core.observation.Observation object at 0x38681a650>, <ax.core.observation.Observation object at 0x386842350>, <ax.core.observation.Observation object at 0x386872690>, <ax.core.observation.Observation object at 0x387c1e6d0>, <ax.core.observation.Observation object at 0x387c2bf90>, <ax.core.observation.Observation object at 0x3867a2210>]
observation_features [ObservationFeatures(parameters={'a': 0.20519250631332397, 'b': 0.9462937712669373}, trial_index=0), ObservationFeatures(parameters={'a': 0.9658348821103573, 'b': 0.3620511395856738}, trial_index=1), ObservationFeatures(parameters={'a': 0.5993889393284917, 'b': 0.6957563571631908}, trial_index=2), ObservationFeatures(parameters={'a': 0.3543400028720498, 'b': 0.11307110730558634}, trial_index=3), ObservationFeatures(parameters={'a': 0.38054878637194633, 'b': 0.5852137189358473}, trial_index=4), Observatio

2024-12-24 23:24:27,962	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-12-24 23:24:27,972	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-24-14' in 0.0093s.
2024-12-24 23:24:28,349	INFO tune.py:1041 -- Total run time: 12.18 seconds (11.77 seconds for the tuning loop).
2024-12-24 23:24:28,349	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-24-14", trainable=...)
2024-12-24 23:24:28,355	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 1 trial(s):
- evaluate_be1f910c: FileNotFoundError('Could not fetch metrics for evaluate_be1f910c: both result.json and prog

ResultGrid<[
  Result(
    metrics={'a': 0.20519250631332397, 'b': 0.9462937712669373},
    path='/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-24-14/evaluate_3851fd8b_1_a=0.2052,b=0.9463_2024-12-24_23-24-16',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'a': 0.9658348821103573, 'b': 0.3620511395856738},
    path='/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-24-14/evaluate_9396dabf_2_a=0.9658,b=0.3621_2024-12-24_23-24-17',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'a': 0.5993889393284917, 'b': 0.6957563571631908},
    path='/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-24-14/evaluate_cda636d3_3_a=0.5994,b=0.6958_2024-12-24_23-24-18',
    filesystem='local',
    checkpoint=None
  ),
  Result(
    metrics={'a': 0.3543400028720498, 'b': 0.11307110730558634},
    path='/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-24-14/evaluate_e747b542_4_a=0.3543,b=0.1131_2024-12-24_23-24-18'

In [1]:
from ax.service.ax_client import AxClient
from ax.service.utils.instantiation import ObjectiveProperties
from ray import tune
from ray.air import session
from ray.tune.search import ConcurrencyLimiter
from axsearch_multiobjective import AxSearchMultiObjective
from ray.train import RunConfig, CheckpointConfig
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=True, download=True, transform=transform),
    batch_size=64,
    shuffle=True,
)
val_loader = torch.utils.data.DataLoader(
    datasets.MNIST("./data", train=False, transform=transform), batch_size=64
)

# Define the evaluation function
def evaluate(parameter: dict, checkpoint_dir=None):
    """
    Evaluate a neural architecture based on given parameters.
    This function trains and validates the model, returning accuracy and latency.
    """
    # Extract parameters
    num_layers = int(parameter["num_layers"])
    hidden_size = int(parameter["hidden_size"])
    kernel_size = int(parameter["kernel_size"])

    # Define a simple CNN architecture
    class SimpleCNN(nn.Module):
        def __init__(self, num_layers, hidden_size, kernel_size):
            super(SimpleCNN, self).__init__()
            layers = []
            in_channels = 1  # Input channels for MNIST (grayscale images)
            for _ in range(num_layers):
                layers.append(
                    nn.Conv2d(in_channels, hidden_size, kernel_size, stride=1, padding=1)
                )
                layers.append(nn.ReLU())
                in_channels = hidden_size

            self.conv = nn.Sequential(*layers)

            # Calculate the output size after convolutions
            conv_output_size = 28  # Initial image size
            for _ in range(num_layers):
                conv_output_size = (conv_output_size - kernel_size + 2 * 1) + 1

            flattened_size = hidden_size * (conv_output_size ** 2)

            self.fc = nn.Linear(flattened_size, 10)  # Fully connected layer for 10 classes

        def forward(self, x):
            x = self.conv(x)
            x = x.view(x.size(0), -1)  # Flatten for the Linear layer
            x = self.fc(x)
            return x

    # Initialize model, criterion, and optimizer
    model = SimpleCNN(num_layers, hidden_size, kernel_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model for 1 epoch (you can extend this for multiple epochs)
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if batch_idx > 50:  # Limit batches for quick tuning
            break
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    # Evaluate on validation data
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            _, predicted = output.max(1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    accuracy = correct / total
    latency = num_layers * hidden_size * kernel_size  # Mock latency (use real metric in production)

    # Report results to Ray Tune
    session.report({"accuracy": accuracy, "latency": latency})


# AxClient for NAS with MOBO
ax_client = AxClient(verbose_logging=False)
ax_client.create_experiment(
    name="nas_mobo",
    parameters=[
        {"name": "num_layers", "type": "range", "value_type": "int", "bounds": [1, 5]},
        {"name": "hidden_size", "type": "range", "value_type": "int", "bounds": [16, 128]},
        {"name": "kernel_size", "type": "choice", "values": [3, 5, 7]},
    ],
    objectives={
        "accuracy": ObjectiveProperties(minimize=False, threshold=0.8),
        "latency": ObjectiveProperties(minimize=True, threshold=1000),
    },
    overwrite_existing_experiment=True,
    is_test=False,
)

# Use AxSearchMultiObjective for search
algo = AxSearchMultiObjective(ax_client=ax_client)
algo = ConcurrencyLimiter(algo, max_concurrent=1)  # Limit to sequential trials

# Tuner for NAS
num_samples = 20  # Number of trials
tuner = tune.Tuner(
    tune.with_resources(evaluate, resources={"cpu": 2, "gpu": 0}),
    tune_config=tune.TuneConfig(search_alg=algo, num_samples=num_samples),
    run_config=RunConfig(
        checkpoint_config=CheckpointConfig(
            num_to_keep=2,
            checkpoint_score_attribute="accuracy",
            checkpoint_score_order="max",
        )
    ),
)

# Run NAS tuning
result_grid = tuner.fit()

observations []
observation_features []
Suggested config: {'num_layers': 1, 'hidden_size': 87, 'kernel_size': 3}


/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))


(raylet) Warning: The actor ImplicitFunc is very large (52 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
Result: {'accuracy': 0.8761, 'latency': 261, 'timestamp': 1735083625, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': 'eec15173', 'date': '2024-12-24_23-40-25', 'time_this_iter_s': 3.469562292098999, 'time_total_s': 3.469562292098999, 'pid': 1499, 'hostname': 'Sidharrths-MacBook-Pro-142.local', 'node_ip': '127.0.0.1', 'time_since_restore': 3.469562292098999, 'iterations_since_restore': 1, 'experiment_tag': '1_hidden_size=87,kernel_size=3,num_layers=1', 'config/num_layers': 1, 'config/hidden_size': 87, 'config/kernel_size': 3}
Metrics to include: ['accuracy', 'latency']
Metric dict after trial: {'accuracy': (0.8761, None), 'latency': (261, None)}
observations [<ax.core.observation.Observation object at 0x31eecf0d0>]
observation_feature

/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/ax/modelbridge/cross_validation.py:463: UserWarning: Encountered exception in computing model fit quality: RandomModelBridge does not support prediction.
  warn("Encountered exception in computing model fit quality: " + str(e))


Result: {'accuracy': 0.898, 'latency': 525, 'timestamp': 1735083638, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': '29ae04fb', 'date': '2024-12-24_23-40-38', 'time_this_iter_s': 10.425843000411987, 'time_total_s': 10.425843000411987, 'pid': 1599, 'hostname': 'Sidharrths-MacBook-Pro-142.local', 'node_ip': '127.0.0.1', 'time_since_restore': 10.425843000411987, 'iterations_since_restore': 1, 'experiment_tag': '2_hidden_size=21,kernel_size=5,num_layers=5', 'config/num_layers': 5, 'config/hidden_size': 21, 'config/kernel_size': 5}
Metrics to include: ['accuracy', 'latency']
Metric dict after trial: {'accuracy': (0.898, None), 'latency': (525, None)}
observations [<ax.core.observation.Observation object at 0x31eb10fd0>, <ax.core.observation.Observation object at 0x3cb47c810>]
observation_features [ObservationFeatures(parameters={'num_layers': 0.09999839999359994, 'hidden_size': 0.632743386326263, 'kernel_size': 0.16666444442962952}, trial_index=0), Observati

...

/Users/sidharrthnagappan/.virtualenvs/lsdp_miniproject/lib/python3.11/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


observations [<ax.core.observation.Observation object at 0x319a272d0>, <ax.core.observation.Observation object at 0x319a57350>, <ax.core.observation.Observation object at 0x319a22990>, <ax.core.observation.Observation object at 0x319a8ca90>, <ax.core.observation.Observation object at 0x3cb52e890>, <ax.core.observation.Observation object at 0x3199a5c10>, <ax.core.observation.Observation object at 0x3199a6050>, <ax.core.observation.Observation object at 0x319e6d0d0>, <ax.core.observation.Observation object at 0x319e6f6d0>, <ax.core.observation.Observation object at 0x3199a4050>, <ax.core.observation.Observation object at 0x319e50490>, <ax.core.observation.Observation object at 0x319e3f890>, <ax.core.observation.Observation object at 0x319a653d0>, <ax.core.observation.Observation object at 0x319ebabd0>, <ax.core.observation.Observation object at 0x319eb9c50>, <ax.core.observation.Observation object at 0x319eb9910>, <ax.core.observation.Observation object at 0x319ed11d0>, <ax.core.observat

2024-12-24 23:45:16,131	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/sidharrthnagappan/ray_results/evaluate_2024-12-24_23-40-16' in 0.0172s.


Result: {'accuracy': 0.9285, 'latency': 504, 'timestamp': 1735083916, 'checkpoint_dir_name': None, 'done': True, 'training_iteration': 1, 'trial_id': '01c12111', 'date': '2024-12-24_23-45-16', 'time_this_iter_s': 8.429319858551025, 'time_total_s': 8.429319858551025, 'pid': 4895, 'hostname': 'Sidharrths-MacBook-Pro-142.local', 'node_ip': '127.0.0.1', 'time_since_restore': 8.429319858551025, 'iterations_since_restore': 1, 'experiment_tag': '20_hidden_size=36,kernel_size=7,num_layers=2', 'config/num_layers': 2, 'config/hidden_size': 36, 'config/kernel_size': 7}
Metrics to include: ['accuracy', 'latency']
Metric dict after trial: {'accuracy': (0.9285, None), 'latency': (504, None)}


2024-12-24 23:45:16,136	INFO tune.py:1041 -- Total run time: 298.48 seconds (298.18 seconds for the tuning loop).


In [4]:
result_grid.get_dataframe()

,accuracy,latency,timestamp,checkpoint_dir_name,done,training_iteration,trial_id,date,time_this_iter_s,time_total_s,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/num_layers,config/hidden_size,config/kernel_size,logdir
0,0.8761,261,1735083625,None,False,1,eec15173,2024-12-24_23-40-25,3.469562,3.469562,1499,Sidharrths-MacBook-Pro-142.local,127.0.0.1,3.469562,1,1,87,3,eec15173
1,0.8980,525,1735083638,None,False,1,29ae04fb,2024-12-24_23-40-38,10.425843,10.425843,1599,Sidharrths-MacBook-Pro-142.local,127.0.0.1,10.425843,1,5,21,5,29ae04fb
2,0.9125,2540,1735083717,None,False,1,58fa2603,2024-12-24_23-41-57,75.922501,75.922501,1742,Sidharrths-MacBook-Pro-142.local,127.0.0.1,75.922501,1,4,127,5,58fa2603
3,0.9276,756,1735083733,None,False,1,dc80ba9d,2024-12-24_23-42-13,12.941924,12.941924,2620,Sidharrths-MacBook-Pro-142.local,127.0.0.1,12.941924,1,2,54,7,dc80ba9d
4,0.8984,2163,1735083779,None,False,1,305570eb,2024-12-24_23-42-59,43.591635,43.591635,2809,Sidharrths-MacBook-Pro-142.local,127.0.0.1,43.591635,1,3,103,7,305570eb
5,0.9171,780,1735083807,None,False,1,c3ba019f,2024-12-24_23-43-27,25.010423,25.010423,3378,Sidharrths-MacBook-Pro-142.local,127.0.0.1,25.010423,1,4,65,3,c3ba019f
6,0.8709,153,1735083813,None,False,1,7e725cc6,2024-12-24_23-43-33,2.038176,2.038176,3721,Sidharrths-MacBook-Pro-142.local,127.0.0.1,2.038176,1,1,51,3,7e725cc6
7,0.8688,240,1735083824,None,False,1,14c4df33,2024-12-24_23-43-44,7.117346,7.117346,3793,Sidharrths-MacBook-Pro-142.local,127.0.0.1,7.117346,1,5,16,3,14c4df33
8,0.9152,112,1735083830,None,False,1,81dd0df6,2024-12-24_23-43-50,1.354536,1.354536,3917,Sidharrths-MacBook-Pro-142.local,127.0.0.1,1.354536,1,1,16,7,81dd0df6
9,0.8870,144,1735083837,None,False,1,5a76f217,2024-12-24_23-43-57,4.162961,4.162961,3977,Sidharrths-MacBook-Pro-142.local,127.0.0.1,4.162961,1,3,16,3,5a76f217


In [7]:
from ax.service.utils.report_utils import _pareto_frontier_scatter_2d_plotly

_pareto_frontier_scatter_2d_plotly(ax_client.experiment)

[WARNING 12-24 23:47:24] ax.service.utils.report_utils: Column reason missing for all trials. Not appending column.


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'line': {'shape': 'hv'},
              'marker': {'color': 'rgba(128,177,211,1)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': array([0.9407, 0.9407, 0.9285, 0.9184, 0.8978, 0.8   ]),
              'y': array([1000.,  602.,  504.,   80.,   48.,   48.])},
             {'marker': {'color': 'rgba(128,177,211,1)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': [0.9407, 0.8],
              'y': [1000.0, 1000.0]},
             {'marker': {'color': 'rgba(128,177,211,1)'},
              'mode': 'lines',
              'type': 'scatter',
              'x': [0.8, 0.8],
              'y': [48.0, 1000.0]},
             {'hovertemplate': '%{text}',
              'marker': {'color': array([  0.,   5.,  10.,  15.,  20.,  25.,  30.,  35.,  40.,  45.,  50.,  55.,
                                          60.,  65.,  70.,  75.,  80.,  85.,  90.,  95., 100.]),
                         'colorbar': {'ticktext': [1, 'iteration', 20], 'tickvals': [0, 50, 100]},
                         'colorscale': [[0.0, '#000004'], [0.1111111111111111,
                                        '#180f3d'], [0.2222222222222222,
                                        '#440f76'], [0.3333333333333333,
                                        '#721f81'], [0.4444444444444444,
                                        '#9e2f7f'], [0.5555555555555556,
                                        '#cd4071'], [0.6666666666666666,
                                        '#f1605d'], [0.7777777777777778,
                                        '#fd9668'], [0.8888888888888888,
                                        '#feca8d'], [1.0, '#fcfdbf']]},
              'mode': 'markers',
              'name': 'Experimental points',
              'text': [Arm name: 0_0, Arm name: 1_0, Arm name: 2_0, Arm name: 3_0,
                       Arm name: 4_0, Arm name: 5_0, Arm name: 6_0, Arm name: 7_0,
                       Arm name: 8_0, Arm name: 9_0, Arm name: 10_0, Arm name:
                       11_0, Arm name: 12_0, Arm name: 13_0, Arm name: 14_0, Arm
                       name: 15_0, Arm name: 16_0, Arm name: 17_0, Arm name: 18_0,
                       Arm name: 19_0],
              'type': 'scatter',
              'x': array([0.8761, 0.898 , 0.9125, 0.9276, 0.8984, 0.9171, 0.8709, 0.8688, 0.9152,
                          0.887 , 0.9184, 0.8991, 0.8978, 0.9101, 0.9138, 0.8815, 0.9407, 0.8833,
                          0.8932, 0.9285]),
              'y': array([ 261.,  525., 2540.,  756., 2163.,  780.,  153.,  240.,  112.,  144.,
                            80.,  160.,   48.,  259.,  322.,  192.,  602.,  384.,   96.,  504.])},
             {'marker': {'color': 'rgba(128,177,211,1)', 'size': 25, 'symbol': 'star'},
              'mode': 'markers',
              'type': 'scatter',
              'x': [0.8],
              'y': [1000.0]}],
    'layout': {'showlegend': False,
               'template': '...',
               'title': {'text': 'Observed metric values with Pareto frontier'},
               'xaxis': {'range': [0.78593, 0.95477], 'title': {'text': 'accuracy'}},
               'yaxis': {'range': [-47.2, 1095.2], 'title': {'text': 'latency'}}}
})

In [6]:
!pip install --upgrade nbformat

  Using cached nbformat-5.10.4-py3-none-any.whl.metadata (3.6 kB)
  Using cached fastjsonschema-2.21.1-py3-none-any.whl.metadata (2.2 kB)
Using cached nbformat-5.10.4-py3-none-any.whl (78 kB)
Using cached fastjsonschema-2.21.1-py3-none-any.whl (23 kB)
  Attempting uninstall: nbformat
    Found existing installation: nbformat 4.2.0
    Uninstalling nbformat-4.2.0:
      Successfully uninstalled nbformat-4.2.0
